# Data cleaning

- Get dataframe of texts from Academy urls 
- Create test corpus to build cleaning function
- Perform first round of data cleaning
    - unwanted symbols
    - make lowercase
    - remove numbers
- Second round of cleaning
    - lemmatisation
    - tokenization
    - remove stop words
- Third round of cleaning
    - stemming
    - create bigrams
- BoW
    
### Once the cleaning steps are combined in a function, we'll run it on the full df of seperated content (i.e. content split by html elements) and full article df (i.e. non-split content)

## 1. Import Academy texts dataframe

In [ ]:
import pandas as pd
import pickle


In [ ]:
df = pd.read_csv('../04_Data/processed_posts.csv', index_col=0)


## 2. Create a subset as the corpus for testing

For now we only need the url and content columns.

In [37]:
df_subset = df[['url','content']].head(10)


In [38]:
df_subset


,url,content
0,https://plana.earth/academy/invest-in-sustaina...,"November is already here, and we are thinking ..."
1,https://plana.earth/academy/invest-in-sustaina...,Plan A is releasing a special offer for its Ca...
2,https://plana.earth/academy/invest-in-sustaina...,How to ensure a sustainable future for busines...
3,https://plana.earth/academy/invest-in-sustaina...,We are on the verge of a historical moment. Th...
4,https://plana.earth/academy/invest-in-sustaina...,Assume this: We have lost 20 million hectares ...
5,https://plana.earth/academy/invest-in-sustaina...,Building a sustainable company gives you a sus...
6,https://plana.earth/academy/invest-in-sustaina...,"Considering this environmental emergency, we b..."
7,https://plana.earth/academy/invest-in-sustaina...,How to build a sustainable company with the Pl...
8,https://plana.earth/academy/invest-in-sustaina...,Start with a clean slate this year. To support...
9,https://plana.earth/academy/invest-in-sustaina...,Why? Our software combines high-end technology...


## 3. Data cleaning round 1
Converting to lower case, get rid of punctuation and numbers

In [ ]:
import re
import string


In [39]:
test = df_subset.loc[9].content
test


'Why? Our software combines high-end technology, science, and fair pricing. Plan A is your quickest way to get a head start on sustainability, regrouping all your data and green activities under one roof. Enterprise customers have been using our tool. Now ambitious startups also have a way to move the needle. Act early in the name of sustainability, Get ahead of your competitors and benefit from expertise in the sustainability field.'

In [ ]:
def cleaning_round1(text):
    '''lowercase, remove punctuation, remove \xa0, remove numbers + words with numbers'''
    
    text = text.lower()
    text = re.sub('-', ' ', text)
    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[”“–‘’]', '', text )
    
    return text


In [ ]:
test_round1 = cleaning_round1(test)
test_round1


## 4. Data cleaning round 2!

The big guns are coming out: lemmatisation, tokenization, stopword removal.

### 4.1 Using lemmatization to reduce words to their root words

In [ ]:
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')


In [ ]:
# lemmatisation

def lemmatizer(text):
    nlp = spacy.load('en_core_web_sm')

    text_out = []
    tokens = nlp(text)
    text_out = [" ".join(token.lemma_ for token in tokens)]
    
    text_out = [re.sub('-PRON-', 'i', str(text)) for text in text_out]
    
    return text_out


In [ ]:
test_lem = lemmatizer(test_round1)
test_lem


### 4.2 Tokenizing and removing stop words

In [ ]:
import gensim
from gensim.utils import simple_preprocess


In [ ]:
def remove_stopwords(texts):
    out = [[word for word in simple_preprocess(str(doc))
            if word not in stop_words]
            for doc in texts]
    return out


In [ ]:
test_no_stp = remove_stopwords(test_lem)
test_no_stp


## 5. Data cleaninfg round 3!

Stemming and bigrams

### 5.1 Stemming with nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps  = PorterStemmer()


In [ ]:
def stemmer(content):
    ps  = PorterStemmer()

    stemmed = [ps.stem(w) for w in content]
    
    return stemmed


In [ ]:
test_stem = stemmer(test_no_stp[0])
test_stem


### 5.2 Creating combined NLP cleaner function

In [ ]:
# This step should be run first to update the df, so we can see how the
# content has been transformed by the cleaning

def nlp_cleaner(content):
    text = cleaning_round1(content)
    text = lemmatizer(text)
    text = remove_stopwords(text)
    text = stemmer(text[0])

    return text


In [ ]:
test_clean = nlp_cleaner(test)
test_clean


### 5.3 Bigrams

Making this part of the get corpus function

In [ ]:
# building bigram models

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    return bigram_mod


In [40]:
# won't create bigrams for single text, but this is to test output

test_bigram = bigrams(test_clean)
print(test_bigram[test_clean])


['softwar', 'combin', 'high', 'end', 'technolog', 'scienc', 'fair', 'price', 'plan', 'quick', 'way', 'get', 'head', 'start', 'sustain', 'regroup', 'datum', 'green', 'activ', 'one', 'roof', 'enterpris', 'custom', 'use', 'tool', 'ambiti', 'startup', 'also', 'way', 'move', 'needl', 'act', 'earli', 'name', 'sustain', 'get', 'ahead', 'competitor', 'benefit', 'expertis', 'sustain', 'field']


## 6. Get cleaned df
Before creating the corpus, it would be helpful to have a cleaned df (inlc. bigrams)

In [ ]:
def clean_df(df):

    df['content'] = df['content'].apply(nlp_cleaner)
    df['content'] = df['content'].apply(lambda x:' '.join(x))
    
    return df 


In [ ]:
df_clean = clean_df(df_subset)
df_clean


## 7. Get the corpus

Combining all data-cleaning steps to create corpus and BoW for LDA model

In [ ]:
def get_corpus(df):
    
    words = list((df.content))
    words = [[word for word in nlp_cleaner(doc)]
            for doc in words]
    
    bigram_mod = bigrams(words)
    bigram_set = [bigram_mod[article] for article in words]
    
    id2word = gensim.corpora.Dictionary(bigram_set)
    id2word.compactify()
    
    corpus = [id2word.doc2bow(text) for text in bigram_set]

    return corpus, id2word, bigram_set


In [ ]:
corpus, id2word, train_bigram = get_corpus(df_subset)


In [ ]:
# check if texts are clean

train_bigram


# 8. Transforming success_urls for LDA model

Using success_urls as it was previously tested. It already went through some cleaning steps.


In [ ]:
import pickle

In [ ]:
with open('../02_LDA_model/pkl_cellar/success_urls.pkl', 'rb') as file:
    success_urls = pickle.load(file)
    
success_urls

In [ ]:
# bigrams are not part of the clean_df process

# success_clean = clean_df(success_urls)
# success_clean

In [ ]:
# Takes 10 minutes to run

succ_corpus, succ_id2word, succ_train_bigram = get_corpus(success_urls)

In [41]:
succ_train_bigram

[['proud',
  'present',
  'major',
  'updat',
  'softwar',
  'product',
  'plan',
  'carbon',
  'manag',
  'sinc',
  'launch',
  'march',
  'singl',
  'larg',
  'upgrad',
  'success',
  'push'],
 ['make',
  'seriou',
  'upgrad',
  'help',
  'measur',
  'accur',
  'carbon_footprint',
  'get',
  'well',
  'insight',
  'enhanc',
  'deci',
  'make',
  'spend',
  'lot',
  'time',
  'think',
  'could',
  'make',
  'journey',
  'sustain',
  'easi',
  'clariti',
  'depth',
  'simplic',
  'three',
  'word',
  'guid',
  'work'],
 ['want',
  'get',
  'inform',
  'book',
  'demo',
  'call',
  'discov',
  'plan',
  'carbon',
  'manag',
  'compani'],
 ['carbon', 'account', 'busi'],
 ['emiss',
  'dashboard',
  'add',
  'depth',
  'context',
  'carbon_footprint',
  'display',
  'carbon_footprint',
  'global',
  'per',
  'employ',
  'track',
  'progress',
  'busi',
  'want',
  'get',
  'snapshot',
  'explain',
  'nathan',
  'bonnisseau',
  'chief',
  'market',
  'offic',
  'plan',
  'particular',
  'em

In [ ]:
#with open('pkl_cellar/new_success_corpus.pkl', 'wb') as sc:
#    pickle.dump(succ_corpus, sc, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#with open('pkl_cellar/new_success_id2word.pkl', 'wb') as si:
#    pickle.dump(succ_id2word, si, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#with open('pkl_cellar/new_success_train_bigram.pkl', 'wb') as st:
#    pickle.dump(succ_train_bigram, st, protocol=pickle.HIGHEST_PROTOCOL)